In [1]:
PRFX = "transformer_20210307C1"
PRFX_PRP = 'prep_20210307B1'

In [2]:
!nvidia-smi

Sun Mar  7 17:49:16 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.100      Driver Version: 440.100      CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:1E.0 Off |                    0 |
| N/A   35C    P0    24W / 300W |     11MiB / 16160MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

- https://colab.research.google.com/drive/1F_RNcHzTfFuQf-LeKvSlud6x7jXYkG31#scrollTo=goRmGIRI5cfC

# setup

http://localhost:8080/notebooks/git/product-category/notebooks/prep_20210304A1.ipynb

In [3]:
HOME = "/data/git/product-category"
p_out = f'{HOME}/data/{PRFX}'
!mkdir -p {p_out}

In [4]:
sz = int(1e4)
DATA2USE = f'{HOME}/data/data_sample_{sz}__{PRFX_PRP}.csv'

# eda 

In [5]:
import pandas as pd
import numpy as np
from collections import Counter

In [6]:
%%time
# df = pd.read_csv(f'../data/data__{PRFX_PRP}.csv')
# df = pd.read_csv(f'{HOME}/data/data_sample__{PRFX_PRP}.csv')
df = pd.read_csv(DATA2USE, nrows=1000)
print(df.shape)
df.sample(3)

(1000, 3)
CPU times: user 8.43 ms, sys: 37 µs, total: 8.47 ms
Wall time: 6.83 ms


,category,title,is_validation
331,Home & Kitchen|Home Dcor|Mirrors|Mirror Sets,Eastland 16&quot; Round Beveled Centerpiece Ta...,1
772,"Patio, Lawn & Garden|Outdoor Power Tools|Repla...","AZ Patio Heaters Natural Gas Hose, Quick Conne...",0
255,"Clothing, Shoes & Jewelry|Girls|Clothing|Sleep...",AME Sleepwear Little Girls' Bubble Guppies Ni...,1


In [7]:
df = pd.read_csv(DATA2USE, nrows=1000)


In [8]:
MIN_CNT = 50
cat2cnt = Counter((j for i in df.category.apply(lambda x: x.split('|')) for j in i))
i2cat = sorted(k for k,v in cat2cnt.items() if v>MIN_CNT)
cat2i = {v:k for k,v in enumerate(i2cat)}

print("len(i2cat)", len(i2cat))
print("|".join(i2cat))

len(i2cat) 10
Accessories|Automotive|Books|Clothing|Clothing, Shoes & Jewelry|Electronics|Home & Kitchen|Sports & Outdoors|Toys & Games|Women


In [9]:
cat2cnt.most_common(20)

[('Books', 213),
 ('Clothing, Shoes & Jewelry', 171),
 ('Women', 103),
 ('Home & Kitchen', 89),
 ('Clothing', 83),
 ('Sports & Outdoors', 64),
 ('Automotive', 64),
 ('Accessories', 63),
 ('Electronics', 61),
 ('Toys & Games', 52),
 ('Men', 47),
 ('Kindle Store', 41),
 ('Kindle eBooks', 40),
 ('Tools & Home Improvement', 40),
 ('Shoes', 40),
 ('Sports & Fitness', 35),
 ('Cell Phones & Accessories', 34),
 ('Replacement Parts', 33),
 ('Imported', 32),
 ('Patio, Lawn & Garden', 28)]

# dataset

In [10]:
from argparse import ArgumentParser
import pytorch_lightning as pl
from transformers.optimization import AdamW

from torch.utils.data import TensorDataset, DataLoader
from transformers import AutoTokenizer
import torch

def mk_tensors(txt, tokenizer, max_seq_length):
    tok_res = tokenizer(
        txt, truncation=True, padding='max_length', max_length=max_seq_length
    )
    input_ids = tok_res["input_ids"]
    attention_mask = tok_res["attention_mask"]
    input_ids = torch.tensor(input_ids)
    attention_mask = torch.tensor(attention_mask)
    return input_ids, attention_mask

def mk_ds(txt, tokenizer, max_seq_length, ys):
    input_ids, attention_mask = mk_tensors(txt, tokenizer, max_seq_length)
    return TensorDataset(input_ids, 
                         attention_mask, 
                         torch.tensor(ys)) 

class PCDataModule(pl.LightningDataModule):
    def __init__(self, 
                 model_name_or_path, 
                 max_seq_length, 
                 min_products_for_category,
                 train_batch_size,
                 val_batch_size,
                 dataloader_num_workers,
                 data_file_path=None,
                 dataframe=None):
        super().__init__()
        self.data_file_path = data_file_path
        self.dataframe = dataframe
        self.min_products_for_category = min_products_for_category
        self.model_name_or_path = model_name_or_path
        self.max_seq_length = max_seq_length
        self.train_batch_size = train_batch_size
        self.val_batch_size = val_batch_size
        self.dataloader_num_workers = dataloader_num_workers
        self.num_classes = None
      
    def prepare_data(self):
        #prepare_data is called from a single process (e.g. GPU 0). Do not use it to assign state (self.x = y).
        _ = AutoTokenizer.from_pretrained(self.model_name_or_path)

    def setup(self, stage=None):
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name_or_path)
        if self.dataframe is None:
            self.dataframe = pd.read_csv(self.data_file_path)
            self.dataframe.dropna(inplace=True)

        
#         self.dataframe = self.dataframe[self.dataframe.title.notna()].copy()
        
        cats = self.dataframe.category.apply(lambda x: x.split('|'))
        cat2cnt = Counter((j for i in cats for j in i))
        i2cat = sorted(k for k,v in cat2cnt.items() if v>self.min_products_for_category)
        cat2i = {v:k for k,v in enumerate(i2cat)}
        self.num_classes = len(i2cat)
        self.i2cat, self.cat2i = i2cat, cat2i
        
        ys = np.zeros((len(self.dataframe), len(i2cat)))
        for i,cats in enumerate(self.dataframe.category):
            idx_pos = [cat2i[cat] for cat in cats.split('|') if cat in cat2i]
            ys[i,idx_pos] = 1
        
        msk_val = self.dataframe.is_validation==1
        self.df_trn = self.dataframe[~msk_val]
        self.df_val = self.dataframe[ msk_val]
        idx_trn = np.where(~msk_val)[0]
        idx_val = np.where( msk_val)[0]
        self.ys_trn, self.ys_val = ys[idx_trn], ys[idx_val]
        
        self.train_dataset = mk_ds(list(self.df_trn.title), self.tokenizer, self.max_seq_length, self.ys_trn)
        self.eval_dataset  = mk_ds(list(self.df_val.title), self.tokenizer, self.max_seq_length, self.ys_val)
        
    
    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.train_batch_size,
            num_workers=self.dataloader_num_workers,
        )

    def val_dataloader(self):
        return DataLoader(
            self.eval_dataset,
            batch_size=self.val_batch_size,
            num_workers=self.dataloader_num_workers,
        )

# model

In [11]:
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModel


def getaccu(logits, ys):
    return ((logits>0.).int() == ys).float().mean()

class PCModel(pl.LightningModule):
    def __init__(self, model_name_or_path, freeze_bert, num_classes, learning_rate, adam_beta1, adam_beta2, adam_epsilon):
        super().__init__()
        self.save_hyperparameters()
        self.model_name_or_path = model_name_or_path
        self.bert = AutoModel.from_pretrained(self.model_name_or_path)
        self.freeze_bert = freeze_bert
        if freeze_bert==True:
            for param in self.bert.parameters():
                param.requires_grad = False
        self.num_classes = num_classes
        self.W = nn.Linear(self.bert.config.hidden_size, self.num_classes)

    def prepare_data(self):
        #prepare_data is called from a single process (e.g. GPU 0). Do not use it to assign state (self.x = y).
        _ = AutoModel.from_pretrained(self.model_name_or_path)

    def forward(self, input_ids, attention_mask):
        h = self.bert(input_ids, attention_mask)['last_hidden_state']
        h_cls = h[:, 0]
        return self.W(h_cls)
    
    def training_step(self, batch, batch_idx):
        input_ids, attention_mask, ys = batch    
        logits = self(input_ids, attention_mask)
        loss = F.binary_cross_entropy_with_logits(logits, ys)
        accu = getaccu(logits, ys)
        self.log('train_loss', loss, on_epoch=True)
        self.log('train_accu', accu, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        input_ids, attention_mask, ys = batch    
        logits = self(input_ids, attention_mask)
        loss = F.binary_cross_entropy_with_logits(logits, ys)
        accu = getaccu(logits, ys)
        self.log('valid_loss', loss, on_step=False, sync_dist=True)
        self.log('valid_accu', accu, on_step=False, sync_dist=True)
    
    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(),
                          self.hparams.learning_rate,
                          betas=(self.hparams.adam_beta1,
                                 self.hparams.adam_beta2),
                          eps=self.hparams.adam_epsilon,)
        return optimizer    
    
    @staticmethod
    def add_model_specific_args(parent_parser):
        parser = ArgumentParser(parents=[parent_parser], add_help=False)
        parser.add_argument('--learning_rate', type=float, default=5e-5)
        parser.add_argument('--adam_beta1', type=float, default=0.9)
        parser.add_argument('--adam_beta2', type=float, default=0.999)
        parser.add_argument('--adam_epsilon', type=float, default=1e-8)
        return parser

# train

In [12]:
parser = ArgumentParser()

parser.add_argument('--model_name_or_path', type=str,
                    default="distilbert-base-cased")
parser.add_argument('--freeze_bert', action='store_true')
parser.add_argument('--max_seq_length', type=int, default=128)
parser.add_argument('--min_products_for_category', type=int, default=100)
parser.add_argument('--train_batch_size', type=int, default=64)
parser.add_argument('--val_batch_size', type=int, default=128)
parser.add_argument("--dataloader_num_workers", type=int, default=8)

parser = pl.Trainer.add_argparse_args(parser)
parser = PCModel.add_model_specific_args(parser)

args = parser.parse_args([
    '--default_root_dir', p_out,
    '--freeze_bert',
])


data_module = PCDataModule(
    model_name_or_path=args.model_name_or_path,
#     data_file_path=f'{HOME}/data/data_sample__{PRFX_PRP}.csv',
    data_file_path=DATA2USE,
    min_products_for_category=args.min_products_for_category,
    max_seq_length=args.max_seq_length,
    train_batch_size=args.train_batch_size,
    val_batch_size=args.val_batch_size,
    dataloader_num_workers=args.dataloader_num_workers,
)

data_module.prepare_data()

In [13]:
%%time
data_module.setup()

CPU times: user 1.72 s, sys: 248 ms, total: 1.97 s
Wall time: 731 ms


In [14]:
pcmodel = PCModel(
    model_name_or_path=args.model_name_or_path,
    freeze_bert=args.freeze_bert,
    num_classes= data_module.num_classes,
    learning_rate=args.learning_rate,
    adam_beta1=args.adam_beta1,
    adam_beta2=args.adam_beta2,
    adam_epsilon=args.adam_epsilon,
)
pcmodel.prepare_data()

## trainer

In [15]:
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import CSVLogger, TensorBoardLogger

csv_logger = CSVLogger(p_out, name='csv')
tb_logger = TensorBoardLogger(p_out, name='tensorboard')

trainer = pl.Trainer.from_argparse_args(args, 
#                                         limit_train_batches=10, limit_val_batches=5, 
#                                         fast_dev_run=True,
                                        max_epochs=10,
                                        callbacks=[EarlyStopping(monitor='valid_loss')],
                                        stochastic_weight_avg=True,
                                        log_gpu_memory=True, 
                                        gpus=1,
                                        logger=[tb_logger,csv_logger],
                                       )

GPU available: True, used: True
TPU available: None, using: 0 TPU cores


## run

In [18]:
pl.seed_everything(1234)
trainer.fit(pcmodel, data_module)

Global seed set to 1234

  | Name | Type            | Params
-----------------------------------------
0 | bert | DistilBertModel | 65.2 M
1 | W    | Linear          | 37.7 K
-----------------------------------------
37.7 K    Trainable params
65.2 M    Non-trainable params
65.2 M    Total params
260.914   Total estimated model params size (MB)


/data/anaconda3/envs/product-category/lib/python3.7/site-packages/pytorch_lightning/core/step_result.py:148: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  value = torch.tensor(value, device=device, dtype=torch.float)


1

In [19]:
!find $p_out/

/data/git/product-category/data/transformer_20210307C1/
/data/git/product-category/data/transformer_20210307C1/tensorboard_csv
/data/git/product-category/data/transformer_20210307C1/tensorboard_csv/0_0
/data/git/product-category/data/transformer_20210307C1/tensorboard_csv/0_0/checkpoints
/data/git/product-category/data/transformer_20210307C1/tensorboard_csv/0_0/checkpoints/epoch=9-step=1329.ckpt
/data/git/product-category/data/transformer_20210307C1/csv
/data/git/product-category/data/transformer_20210307C1/csv/version_0
/data/git/product-category/data/transformer_20210307C1/csv/version_0/metrics.csv
/data/git/product-category/data/transformer_20210307C1/csv/version_0/hparams.yaml
/data/git/product-category/data/transformer_20210307C1/tensorboard
/data/git/product-category/data/transformer_20210307C1/tensorboard/version_0
/data/git/product-category/data/transformer_20210307C1/tensorboard/version_0/events.out.tfevents.1615139406.ip-10-0-3-91.24318.0
/data/git/product-category/data/trans

## tensorboard

In [22]:
!find $p_out/tensorboard/

/data/git/product-category/data/transformer_20210307C1/tensorboard/
/data/git/product-category/data/transformer_20210307C1/tensorboard/version_0
/data/git/product-category/data/transformer_20210307C1/tensorboard/version_0/events.out.tfevents.1615139406.ip-10-0-3-91.24318.0
/data/git/product-category/data/transformer_20210307C1/tensorboard/version_0/hparams.yaml


## load model

# eval

## csv results

In [25]:
!find $p_out/csv/

/data/git/product-category/data/transformer_20210307C1/csv/
/data/git/product-category/data/transformer_20210307C1/csv/version_0
/data/git/product-category/data/transformer_20210307C1/csv/version_0/metrics.csv
/data/git/product-category/data/transformer_20210307C1/csv/version_0/hparams.yaml


In [26]:
dfmtr = pd.read_csv(f"{p_out}/csv/version_0/metrics.csv")

dfmtr

,train_loss_step,train_accu_step,gpu_id: 0/memory.used (MB),epoch,step,train_loss_epoch,train_accu_epoch,valid_loss,valid_accu
0,0.552723,0.846620,2018.0,0,49,NaN,NaN,NaN,NaN
1,0.429126,0.957589,2018.0,0,99,NaN,NaN,NaN,NaN
2,NaN,NaN,2018.0,0,132,0.521671,0.841016,NaN,NaN
3,NaN,NaN,2546.0,0,132,NaN,NaN,0.364051,0.96202
4,0.350938,0.960778,2546.0,1,149,NaN,NaN,NaN,NaN
5,0.290515,0.963329,2546.0,1,199,NaN,NaN,NaN,NaN
6,0.250279,0.962372,2546.0,1,249,NaN,NaN,NaN,NaN
7,NaN,NaN,2548.0,1,265,0.298023,0.961705,NaN,NaN
8,NaN,NaN,2548.0,1,265,NaN,NaN,0.237499,0.96202
9,0.230249,0.962054,2548.0,2,299,NaN,NaN,NaN,NaN


## run model

## demo

In [31]:
def do_demo(df):
    i2cat = data_module.i2cat
    tokenizer = data_module.tokenizer
    max_seq_length = data_module.max_seq_length
    row = df.sample()
    display(row)
    txt = list(row.title.values)
    input_ids, attention_mask = mk_tensors(txt, tokenizer, max_seq_length)
    logits = pcmodel(input_ids, attention_mask)[0]
    print('Truth:', sorted(o for o in row.category.values[0].split('|') if o in i2cat))

    top_icats = np.argsort(-logits.detach().numpy())[:5]
    print('Top Preds:',[i2cat[i] for i in top_icats])
    print('Preds 0.5+:',[i2cat[i] for i in np.where(logits>0)[0]])


In [32]:
do_demo(data_module.df_val)

,category,title,is_validation
4913,Books|Literature &amp; Fiction|Genre Fiction,My Interview with Beethoven,1


Truth: ['Books', 'Literature &amp; Fiction']
Top Preds: ['Books', 'Clothing, Shoes & Jewelry', 'Women', 'Clothing', 'Home & Kitchen']
Preds 0.5+: []


# fin

In [33]:
!nvidia-smi

Sun Mar  7 17:54:39 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.100      Driver Version: 440.100      CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:1E.0 Off |                    0 |
| N/A   53C    P0    44W / 300W |   1252MiB / 16160MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  